In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import datasets  
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.tree import  DecisionTreeClassifier
from sklearn import tree
from sklearn.metrics import classification_report
from sklearn import preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import BaggingClassifier 
from sklearn.tree import DecisionTreeClassifier 

In [2]:
fraud = pd.read_csv("Fraud_check.csv")
fraud

,Undergrad,Marital.Status,Taxable.Income,City.Population,Work.Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO
...,...,...,...,...,...,...
595,YES,Divorced,76340,39492,7,YES
596,YES,Divorced,69967,55369,2,YES
597,NO,Divorced,47334,154058,0,YES
598,YES,Married,98592,180083,17,NO


In [3]:
# EDA

fraud = fraud.rename({'Marital.Status':'Marital_Status',
                      'Taxable.Income':'Taxable_Income',
                      'City.Population':'City_Population',
                      'Work.Experience':'Work_Experience'},axis=1)
fraud.head()

,Undergrad,Marital_Status,Taxable_Income,City_Population,Work_Experience,Urban
0,NO,Single,68833,50047,10,YES
1,YES,Divorced,33700,134075,18,YES
2,NO,Married,36925,160205,30,YES
3,YES,Single,50190,193264,15,YES
4,NO,Married,81002,27533,28,NO


In [4]:
fraud.shape

(600, 6)

In [5]:
fraud.describe()

,Taxable_Income,City_Population,Work_Experience
count,600.000000,600.000000,600.000000
mean,55208.375000,108747.368333,15.558333
std,26204.827597,49850.075134,8.842147
min,10003.000000,25779.000000,0.000000
25%,32871.500000,66966.750000,8.000000
50%,55074.500000,106493.500000,15.000000
75%,78611.750000,150114.250000,24.000000
max,99619.000000,199778.000000,30.000000


In [6]:
fraud.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 600 entries, 0 to 599
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   Undergrad        600 non-null    object
 1   Marital_Status   600 non-null    object
 2   Taxable_Income   600 non-null    int64 
 3   City_Population  600 non-null    int64 
 4   Work_Experience  600 non-null    int64 
 5   Urban            600 non-null    object
dtypes: int64(3), object(3)
memory usage: 28.3+ KB


In [7]:
fraud.isnull().sum()

Undergrad          0
Marital_Status     0
Taxable_Income     0
City_Population    0
Work_Experience    0
Urban              0
dtype: int64

In [8]:
# We have taxable_income <= 30000 as "Risky" and others are "Good"

fraud.insert(6,'Tax_Class'," ")

In [9]:
for i in range(0,len(fraud['Taxable_Income']),1):
    if fraud['Taxable_Income'].iloc[i]<=30000:
        fraud['Tax_Class'].iloc[i]='Risky'
    else:
        fraud['Tax_Class'].iloc[i]='Good'

C:\Users\vaibh\AppData\Local\Temp\ipykernel_17252\1013303792.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fraud['Tax_Class'].iloc[i]='Good'
C:\Users\vaibh\AppData\Local\Temp\ipykernel_17252\1013303792.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fraud['Tax_Class'].iloc[i]='Risky'


In [10]:
fraud.drop(['Taxable_Income'],axis=1,inplace=True)
fraud.head()

,Undergrad,Marital_Status,City_Population,Work_Experience,Urban,Tax_Class
0,NO,Single,50047,10,YES,Good
1,YES,Divorced,134075,18,YES,Good
2,NO,Married,160205,30,YES,Good
3,YES,Single,193264,15,YES,Good
4,NO,Married,27533,28,NO,Good


In [11]:
fraud.Tax_Class.value_counts()

Good     476
Risky    124
Name: Tax_Class, dtype: int64

In [13]:
labelencoder = LabelEncoder()
fraud.iloc[:, -1] = labelencoder.fit_transform(fraud.iloc[:,-1])

C:\Users\vaibh\AppData\Local\Temp\ipykernel_17252\1079862779.py:2: DeprecationWarning: In a future version, `df.iloc[:, i] = newvals` will attempt to set the values inplace instead of always setting a new array. To retain the old behavior, use either `df[df.columns[i]] = newvals` or, if columns are non-unique, `df.isetitem(i, newvals)`
  fraud.iloc[:, -1] = labelencoder.fit_transform(fraud.iloc[:,-1])


In [14]:
fraud.head()

,Undergrad,Marital_Status,City_Population,Work_Experience,Urban,Tax_Class
0,NO,Single,50047,10,YES,0
1,YES,Divorced,134075,18,YES,0
2,NO,Married,160205,30,YES,0
3,YES,Single,193264,15,YES,0
4,NO,Married,27533,28,NO,0


In [15]:
fraud = pd.get_dummies(fraud, columns=["Undergrad","Marital_Status","Urban"])

In [16]:
fraud.head()

,City_Population,Work_Experience,Tax_Class,Undergrad_NO,Undergrad_YES,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Urban_NO,Urban_YES
0,50047,10,0,1,0,0,0,1,0,1
1,134075,18,0,0,1,1,0,0,0,1
2,160205,30,0,1,0,0,1,0,0,1
3,193264,15,0,0,1,0,0,1,0,1
4,27533,28,0,1,0,0,1,0,1,0


In [17]:
# Normalization function 

def norm_func(i):
    x = (i-i.min())/(i.max()-i.min())
    return (x)

fraud_norm = norm_func(fraud.iloc[:,0:])
fraud_norm.head()

,City_Population,Work_Experience,Tax_Class,Undergrad_NO,Undergrad_YES,Marital_Status_Divorced,Marital_Status_Married,Marital_Status_Single,Urban_NO,Urban_YES
0,0.139472,0.333333,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0
1,0.622394,0.600000,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0
2,0.772568,1.000000,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0
3,0.962563,0.500000,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0
4,0.010081,0.933333,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0


In [18]:
# Random Forest Classification

from sklearn.ensemble import RandomForestClassifier

array = fraud_norm.values
X = fraud_norm.loc[:, fraud_norm.columns != "Tax_Class"]
Y = fraud_norm["Tax_Class"]

num_trees = 200
kfold = KFold(n_splits=10, random_state=7, shuffle=True) 

model = RandomForestClassifier(n_estimators=num_trees,max_depth=5,  min_samples_leaf=5,max_features='sqrt')
results = cross_val_score(model, X, Y, cv=kfold)
print(results)
print(results.mean())

[0.78333333 0.73333333 0.86666667 0.91666667 0.75       0.76666667
 0.83333333 0.66666667 0.83333333 0.78333333]
0.7933333333333332


In [19]:
# Use Grid search CV to find best parameters

from sklearn.model_selection import GridSearchCV
rf = RandomForestClassifier(random_state=42, n_jobs=-1)
params = {
    'max_depth': [2,3,5,None],
    'min_samples_leaf': [5,10,20],
    'n_estimators': [50,100,200,500],
    'max_features':["sqrt","log2",None]
}

# Instantiate the grid search model
grid_search = GridSearchCV(estimator=rf,
                           param_grid=params,
                           cv = 5,
                           n_jobs=-1, verbose=2, scoring="accuracy")  # n_jobs: parallel computing
grid_search.fit(X, Y)

Fitting 5 folds for each of 144 candidates, totalling 720 fits


GridSearchCV(cv=5, estimator=RandomForestClassifier(n_jobs=-1, random_state=42),
             n_jobs=-1,
             param_grid={'max_depth': [2, 3, 5, None],
                         'max_features': ['sqrt', 'log2', None],
                         'min_samples_leaf': [5, 10, 20],
                         'n_estimators': [50, 100, 200, 500]},
             scoring='accuracy', verbose=2)

In [21]:
print(grid_search.best_params_)

{'max_depth': 2, 'max_features': 'sqrt', 'min_samples_leaf': 5, 'n_estimators': 50}


In [22]:
print(grid_search.best_score_)

0.7933333333333332
